In [1]:
import pandas as pd
import os, re
from datetime import datetime

In [2]:
df = pd.read_csv('./Descargados/Datos IPC mensual por provincia.csv', sep = ';')
print(df.shape)
df.head()

(777192, 5)


,Provincias,Grupos ECOICOP,Tipo de dato,Periodo,Total
0,01 Araba/Álava,Índice general,Índice,2025M06,NaN
1,01 Araba/Álava,Índice general,Índice,2025M05,"117,634"
2,01 Araba/Álava,Índice general,Índice,2025M04,"117,475"
3,01 Araba/Álava,Índice general,Índice,2025M03,"116,527"
4,01 Araba/Álava,Índice general,Índice,2025M02,"116,17"


In [3]:
df = df[df.Provincias != 'Nacional'] #Se eliminan filas de valores nacionales
df = df[df['Tipo de dato'] == 'Índice'] #De todos los tipos de datos nos quedamos solo con el índice de IPC
df['Anio'] = df.Periodo.str[:4] #Obtenemos el año del Periodo
df['Mes'] = df.Periodo.str[-2:] #Obtenemos el mes del Periodo
df.Total = df.Total.str.replace(',','.').astype(float) #Convetimos el índice del ipc en numércio decimal
df.Anio = df.Anio.astype(int) #convertimos el año en número entero
df.Mes = df.Mes.astype(int) #Convertimos el mes en número entero
df = df[df.Anio < 2025] #Eliminamos datos del 2025 por estar incompletos y fuera del estudio
df = df.drop(columns = ['Tipo de dato','Periodo']) #Eliminamos columnas innecesarias
df.head()

,Provincias,Grupos ECOICOP,Total,Anio,Mes
6,01 Araba/Álava,Índice general,115.462,2024,12
7,01 Araba/Álava,Índice general,114.842,2024,11
8,01 Araba/Álava,Índice general,114.525,2024,10
9,01 Araba/Álava,Índice general,113.806,2024,9
10,01 Araba/Álava,Índice general,114.691,2024,8


In [4]:
df2 = df.pivot_table(index = ['Provincias','Anio','Mes'],
                     columns = 'Grupos ECOICOP',
                     values = 'Total',
                     aggfunc = 'max').reset_index()
dict_rename = {'Provincias':'Provincia',
               '01 Alimentos y bebidas no alcohólicas':'01_alimentos',
               '02 Bebidas alcohólicas y tabaco':'02_tabaco',
               '03 Vestido y calzado':'03_vestido',
               '04 Vivienda, agua, electricidad, gas y otros combustibles':'04_vivienda',
               '05 Muebles, artículos del hogar y artículos para el mantenimiento corriente del hogar':'05_hogar',
               '06 Sanidad':'06_sanidad',
               '07 Transporte':'07_transporte',
               '08 Comunicaciones':'08_comunicaciones',
               '09 Ocio y cultura':'09_ocio',
               '10 Enseñanza':'10_enseñanza',
               '11 Restaurantes y hoteles':'11_restaurantes',
               '12 Otros bienes y servicios':'12_otros',
               'Índice general':'00_general'}
df2 = df2.rename(columns = dict_rename)
df2.Provincia = df2.Provincia.str[3:]
df2.head()

Grupos ECOICOP,Provincia,Anio,Mes,01_alimentos,02_tabaco,03_vestido,04_vivienda,05_hogar,06_sanidad,07_transporte,08_comunicaciones,09_ocio,10_enseñanza,11_restaurantes,12_otros,00_general
0,Araba/Álava,2002,1,59.846,47.135,79.795,59.323,79.628,82.842,66.706,126.135,100.332,60.959,64.705,64.476,68.415
1,Araba/Álava,2002,2,59.481,47.163,80.307,59.226,79.735,83.103,67.033,124.207,100.294,61.460,65.177,65.178,68.519
2,Araba/Álava,2002,3,59.525,47.075,80.969,59.345,80.797,83.864,67.869,122.637,101.988,61.477,65.903,65.341,68.973
3,Araba/Álava,2002,4,59.888,48.960,85.531,59.393,81.050,84.117,68.801,122.637,101.329,61.588,66.394,65.512,69.740
4,Araba/Álava,2002,5,60.102,49.073,87.084,59.503,81.338,83.773,69.066,122.637,101.755,61.601,66.594,65.813,70.058


In [9]:
df3 = df2.copy()
cols_a_normalizar = df3.columns.difference(['Provincia', 'Anio', 'Mes'])
df3[cols_a_normalizar] = df3.groupby(['Anio', 'Mes'])[cols_a_normalizar].transform(lambda x: (x - x.min()) / (x.max() - x.min()))
df3.head()
  
    

Grupos ECOICOP,Provincia,Anio,Mes,01_alimentos,02_tabaco,03_vestido,04_vivienda,05_hogar,06_sanidad,07_transporte,08_comunicaciones,09_ocio,10_enseñanza,11_restaurantes,12_otros,00_general
0,Araba/Álava,2002,1,0.0,0.329595,0.121225,0.404694,0.259080,0.337675,0.986139,0.419610,0.304733,0.524629,0.474133,0.299607,0.265888
1,Araba/Álava,2002,2,0.0,0.326295,0.206063,0.390564,0.264003,0.334764,0.994097,0.418492,0.299168,0.552204,0.490857,0.339854,0.276932
2,Araba/Álava,2002,3,0.0,0.299644,0.177410,0.380977,0.290382,0.343135,0.987066,0.417606,0.308812,0.550119,0.489952,0.341880,0.270747
3,Araba/Álava,2002,4,0.0,0.389654,0.137287,0.370903,0.284839,0.341899,0.981741,0.417606,0.312622,0.556127,0.497761,0.335455,0.250760
4,Araba/Álava,2002,5,0.0,0.379785,0.150273,0.375693,0.283999,0.341150,0.987084,0.417606,0.330901,0.556783,0.512130,0.348720,0.257586


In [24]:
#df3.to_csv('./Transformados/IPC_provincia_mes_normalizado.csv', sep = ';', index = False)
#print(datetime.now(), ' - guardado')

2025-06-30 23:25:42.237681  - guardado


In [22]:
df4 = df3.copy()
df4['Trimestre'] = ((df4.Mes - 1)/3 + 1).astype(int)
df4 = df4.groupby(['Provincia','Anio','Trimestre']).mean().reset_index()
df4.drop(columns = ['Mes'], inplace = True)
df4.head()

Grupos ECOICOP,Provincia,Anio,Trimestre,01_alimentos,02_tabaco,03_vestido,04_vivienda,05_hogar,06_sanidad,07_transporte,08_comunicaciones,09_ocio,10_enseñanza,11_restaurantes,12_otros,00_general
0,Albacete,2002,1,0.634193,0.063950,0.270878,0.262845,0.259843,0.618676,0.752437,0.298753,0.571441,0.066232,0.291126,0.495533,0.366013
1,Albacete,2002,2,0.619563,0.124565,0.282620,0.244733,0.237801,0.603431,0.752544,0.298884,0.623860,0.061742,0.370126,0.472302,0.366195
2,Albacete,2002,3,0.636933,0.133653,0.328550,0.257351,0.262517,0.609932,0.775474,0.303848,0.629821,0.064018,0.363205,0.506031,0.388300
3,Albacete,2002,4,0.648598,0.127760,0.340773,0.262996,0.236468,0.601704,0.754492,0.306089,0.569657,0.066880,0.373874,0.502312,0.371673
4,Albacete,2003,1,0.612128,0.123472,0.245386,0.296012,0.266691,0.574804,0.768443,0.307849,0.545166,0.053679,0.402681,0.516699,0.352506


In [23]:
df4.to_csv('./Transformados/IPC_provincia_Trimestre_normalizado.csv', sep = ';', index = False)
print(datetime.now(), ' - guardado')

2025-07-06 23:46:19.539064  - guardado
